In [ ]:
# ============================================================
# Подготовка окружения: импорты, устройство, конфиг.
# Этот блок ничего не скачивает и не обучает — только настраивает.
# ============================================================
# --- стандартные библиотеки ---
import os, math, random, time
from dataclasses import dataclass
from typing import List, Tuple, Dict
import pandas as pd # для таблиц
import matplotlib.pyplot as plt  # для графиков
import numpy as np
# --- PyTorch ядро ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Hugging Face Datasets: для загрузки готовых датасетов (изображения, тексты и др.)
from datasets import load_dataset

# TorchVision: стандартные преобразования изображений (resize, crop, normalize и т.д.)
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

# -----------------------------
# Функция для воспроизводимости
# -----------------------------
# ========== ВОСПРОИЗВОДИМОСТЬ ==========
def set_seed(seed: int = 42) -> None:
    """Фиксирует ГСЧ для повторяемых результатов (Python/NumPy/PyTorch)."""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(42)

# -----------------------------
# Определение устройства (CPU/GPU)
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if torch.cuda.is_available():
    # Детерминизм на CUDA (чуть медленнее, но стабильнее)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # При необходимости:
    # torch.use_deterministic_algorithms(True)  


# -----------------------------
# Конфигурация эксперимента
# -----------------------------
@dataclass
class Config:
    # Имя датасета в Hugging Face Datasets; 'beans' — небольшой датасет изображений (3 класса)
    dataset_name: str = "beans"            # str: идентификатор датасета в HF

    # Целевой размер стороны изображения после ресайза (квадрат до image_size x image_size)
    image_size: int = 128                  # int: компромисс скорость/качество; 128 достаточно для быстрых итераций

    # Размер мини-батча: влияет на стабильность градиента и скорость итераций
    batch_size: int = 16                   # int: 32–128 обычно стабильны на небольших моделях/датасетах

    # Количество воркеров для DataLoader (параллельная загрузка данных)
    num_workers: int = 0                   # int: 0 для Windows/ноутбуков без многопроцессной загрузки; 2–4 часто достаточно

    epochs: int = 10                        # Количество эпох обучения (проходов по train-части датасета)

    # Базовая скорость обучения (learning rate) — ключевой гиперпараметр оптимизации
    lr: float = 1e-2                       # float: стартовое значение; оптимальный LR зависит от оптимизатора и модели

    # Коэффициент L2-регуляризации (weight decay): для AdamW используется как «правильный» распад весов
    weight_decay: float = 0.0              # float: >=0; напр. 0.01 для AdamW

    # Использовать ли смешанную точность (AMP) при наличии CUDA (ускорение + экономия памяти)
    mixed_precision: bool = True           # bool: True — включать torch.autocast при обучении на GPU

cfg = Config()
print(cfg)


In [ ]:
# ============================================================
# Загрузка датасета 'beans' с кешированием в ./data/hf_cache
# и формирование одной аккуратной таблицы со сводной информацией.
# ============================================================

set_seed(42)
# 1) Подготовка локальной папки для кеша
os.makedirs("./data/hf_cache", exist_ok=True)

# 2) Загрузка датасета с локальным кешированием
ds = load_dataset(cfg.dataset_name, cache_dir="./data/hf_cache")

# 3) Проверка наличия ожидаемых сплитов
expected_splits = {"train", "validation", "test"}
actual_splits = set(ds.keys())
missing = expected_splits - actual_splits
assert not missing, f"Отсутствуют сплиты: {missing}. Доступные: {actual_splits}"

# 4) Получение ссылок на сплиты
train_ds, val_ds, test_ds = ds["train"], ds["validation"], ds["test"]

# 5) Информация о классах (человеко-читаемые имена)
label_names = train_ds.features["labels"].names
num_classes = len(label_names)

# 6) Формирование одной таблицы: сплит + размер + число классов + список имён классов
info_records = [
    {
        "split": split,
        "size": len(ds[split]),
        "num_classes": num_classes,
        "class_names": ", ".join(label_names)
    }
    for split in sorted(ds.keys())
]

info_df = pd.DataFrame(info_records)

# 7) Оформление таблицы Styler: шапка, выравнивание, форматирование
styled_info = (
    info_df.style
    .set_caption("Сводная информация о датасете 'beans'")
    .format({"size": "{:,}"})
    .hide(axis="index")
    .set_table_styles([
        {"selector": "caption", "props": [("font-size", "16px"), ("font-weight", "bold"), ("text-align", "left")]},
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "td", "props": [("text-align", "center")]},
    ])
)

display(styled_info)

In [ ]:
# ============================================================
# Распределение классов по каждому сплиту
# ============================================================

import pandas as pd  # уже был в 2A; оставлено на случай отдельного исполнения ячейки

def class_distribution(split_name: str) -> pd.DataFrame:
    """
    Возвращает DataFrame с частотами и долями классов для указанного сплита.
    Колонки: class_id, class_name, count, share
    """
    labels = ds[split_name]["labels"]  # список int-меток
    counts = pd.Series(labels).value_counts().sort_index()
    df = pd.DataFrame({
        "class_id": counts.index,
        "class_name": [label_names[i] for i in counts.index],
        "count": counts.values
    })
    df["share"] = df["count"] / df["count"].sum()
    return df
set_seed(42)
train_dist = class_distribution("train")
val_dist   = class_distribution("validation")
test_dist  = class_distribution("test")

def style_distribution(df: pd.DataFrame, title: str) -> pd.io.formats.style.Styler:
    return (
        df.style
        .set_caption(title)
        .format({"count": "{:,}", "share": "{:.2%}"})
        .bar(subset=["share"])  # визуальная полоса по долям
        .hide(axis="index")
        .set_table_styles([
            {"selector": "caption", "props": [("font-size", "16px"), ("font-weight", "bold"), ("text-align", "left")]},
            {"selector": "th", "props": [("text-align", "center")]},
            {"selector": "td", "props": [("text-align", "center")]},
        ])
    )

display(style_distribution(train_dist, "Распределение классов — train"))
display(style_distribution(val_dist,   "Распределение классов — validation"))
display(style_distribution(test_dist,  "Распределение классов — test"))

In [ ]:
# ============================================================
# для каждого класса три столбца (train/validation/test).
# ============================================================

import numpy as np

# Упорядочиваем по class_id, чтобы бары совпадали по позициям
t = train_dist.sort_values("class_id").reset_index(drop=True)
v = val_dist.sort_values("class_id").reset_index(drop=True)
e = test_dist.sort_values("class_id").reset_index(drop=True)

x = np.arange(len(t))      # позиции классов
w = 0.25                   # ширина одного столбца

plt.figure(figsize=(8, 4.5))
plt.bar(x - w, t["count"], width=w, label="train")
plt.bar(x,     v["count"], width=w, label="validation")
plt.bar(x + w, e["count"], width=w, label="test")

plt.xticks(x, t["class_name"], rotation=0)
plt.title("Сравнение распределения классов по сплитам (count)")
plt.xlabel("Класс")
plt.ylabel("Количество")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


### Вывод

Распределение классов в датасете *Beans* является **равномерным** (около 33 % на каждый класс).  
Такое соотношение **исключает смещение модели** и обеспечивает **корректную оценку метрик качества** при обучении и сравнении оптимизаторов.  

Классы:  
- **angular_leaf_spot** — угловатая пятнистость листьев  
- **bean_rust** — ржавчина фасоли  
- **healthy** — здоровые листья


In [ ]:
# ============================================================
# Визуализация сетки примеров изображений.
# Показывает n случайных образцов из указанного сплита.
# ============================================================

def show_image_grid(
    split_name: str = "train",
    n: int = 10,                 # количество изображений в сетке
    cols: int = 5,               # число столбцов в сетке
    seed: int = 42,              # для воспроизводимого выбора
    title: str | None = None,    # заголовок фигуры
) -> None:
    """
    Визуализирует случайную подборку изображений из сплита ds[split_name].
    Подпись под каждым изображением — человеко-читаемое имя класса.
    """
    assert split_name in ds, f"Неизвестный сплит: {split_name}. Доступные: {list(ds.keys())}"
    rng = random.Random(seed)

    # Выбор индексов
    split = ds[split_name]
    n = min(n, len(split))                    # защита от выхода за пределы
    idxs = rng.sample(range(len(split)), n)   # случайные индексы без повторов

    # Параметры сетки
    rows = math.ceil(n / cols)
    plt.figure(figsize=(cols * 2.5, rows * 2.5))  # масштабируем размер фигуры от сетки

    for i, idx in enumerate(idxs, start=1):
        example = split[idx]
        img = example["image"]                # PIL.Image.Image
        label_id = example["labels"]
        label = label_names[label_id]

        ax = plt.subplot(rows, cols, i)
        ax.imshow(img)                        # изображения в beans RGB, без нормализации для просмотра
        ax.set_title(label, fontsize=10)      # краткая подпись класса
        ax.axis("off")                        # скрыть оси для чистоты визуализации

    if title is None:
        title = f"Примеры изображений — {split_name}"
    plt.suptitle(title, y=0.98, fontsize=12)
    plt.tight_layout()
    plt.show()

# Пример вызова: сетка из train-сплита
show_image_grid(split_name="train", n=10, cols=5, seed=42)


In [ ]:
# ============================================================
# Блок 2C.2. Визуализация: по одному примеру на класс.
# Удобно для быстрой проверки классов и их отличий.
# ============================================================

def show_one_per_class(
    split_name: str = "train",
    seed: int = 42,                   # для воспроизводимого выбора среди множества примеров класса
    cols: int | None = None,          # число столбцов; по умолчанию = числу классов (в одну строку)
    title: str | None = None,
) -> None:
    assert split_name in ds, f"Неизвестный сплит: {split_name}. Доступные: {list(ds.keys())}"
    rng = random.Random(seed)
    split = ds[split_name]

    # Для каждого класса берём случайный индекс соответствующего примера
    by_class = {i: [] for i in range(num_classes)}
    for idx in range(len(split)):
        lab = split[idx]["labels"]
        if len(by_class[lab]) < 32:  # ограничиваем накопление для ускорения
            by_class[lab].append(idx)

    chosen = []
    for c in range(num_classes):
        assert len(by_class[c]) > 0, f"В сплите {split_name} нет примеров класса {c} ({label_names[c]})"
        chosen.append(rng.choice(by_class[c]))

    # Сетка: одна строка по умолчанию (или разбивка на несколько строк)
    if cols is None:
        cols = num_classes
    rows = math.ceil(num_classes / cols)

    plt.figure(figsize=(cols * 4, rows * 4))
    for i, idx in enumerate(chosen, start=1):
        ex = split[idx]
        img = ex["image"]
        lab = ex["labels"]
        ax = plt.subplot(rows, cols, i)
        ax.imshow(img)
        ax.set_title(f"{label_names[lab]}", fontsize=10)
        ax.axis("off")

    if title is None:
        title = f"По одному примеру на класс — {split_name}"
    plt.suptitle(title, y=0.98, fontsize=12)
    plt.tight_layout()
    plt.show()

# Пример
show_one_per_class(split_name="train", seed=111, cols=None)


In [ ]:
# ============================================================
# Демонстрация простых преобразований (аугментация):
#   1) Оригинал (как есть)
#   2) Resize до cfg.image_size (качественная интерполяция LANCZOS)
#   3) Resize + горизонтальный флип
# Показ без сглаживания (interpolation="none") для сравнения резкости.
# ============================================================

import torchvision.transforms.functional as TF 
set_seed(42)
def preview_resize_flip(
    split_name: str = "train",
    n: int = 5,               # число строк (изображений) для показа
    seed: int = 42,           # фиксируем выбор образцов
    size: int | None = None,  # целевой размер стороны; по умолчанию cfg.image_size
) -> None:
    assert split_name in ds, f"Неизвестный сплит: {split_name}. Доступные: {list(ds.keys())}"
    rng = random.Random(seed)
    split = ds[split_name]
    n = min(n, len(split))
    if size is None:
        size = cfg.image_size

    # выбираем детерминированно n индексов
    idxs = list(range(len(split)))
    rng.shuffle(idxs)
    idxs = idxs[:n]

    cols = 3  # оригинал | ресайз | ресайз+флип
    plt.figure(figsize=(cols * 3.2, n * 2.6))

    for row, idx in enumerate(idxs):
        ex = split[idx]
        img_pil = ex["image"]                  # PIL.Image
        lab = label_names[ex["labels"]]

        # --- (1) Оригинал ---
        ax = plt.subplot(n, cols, row * cols + 1)
        ax.imshow(img_pil, interpolation="none")  # никаких доп. сглаживаний
        ax.set_title(f"оригинал | {lab}", fontsize=9)
        ax.axis("off")

        # --- (2) Ресайз ---
        img_resized = TF.resize(
            img_pil,
            size=[size, size],
            interpolation=InterpolationMode.LANCZOS,  # качественный даунскейл
            antialias=True
        )
        ax = plt.subplot(n, cols, row * cols + 2)
        ax.imshow(img_resized, interpolation="none")
        ax.set_title(f"ресайз {size}×{size}", fontsize=9)
        ax.axis("off")

        # --- (3) Ресайз + флип ---
        img_flip = TF.hflip(img_resized)
        ax = plt.subplot(n, cols, row * cols + 3)
        ax.imshow(img_flip, interpolation="none")
        ax.set_title(f"ресайз {size}×{size} + флип", fontsize=9)
        ax.axis("off")

    plt.suptitle(f"Простые преобразования — {split_name}", y=0.995, fontsize=12)
    plt.tight_layout()
    plt.show()

# Пример вызова:
preview_resize_flip(split_name="train", n=6, seed=7)


### Вывод

На представленных примерах показаны базовые операции аугментации изображений в обучающей выборке:
1. **Оригинал** — исходное изображение без изменений.  
2. **Resize** — масштабирование до фиксированного размера `128×128` с использованием интерполяции *LANCZOS* для сохранения деталей.  
3. **Resize + Flip** — масштабирование и горизонтальное отражение, что увеличивает вариативность данных.

Такие преобразования повышают **обобщающую способность модели**, предотвращая переобучение и позволяя модели устойчивее распознавать объекты при изменении ориентации и масштаба.


In [ ]:
# ------------------------------------------------------------
# Преобразования входных изображений.
# Обучение: Resize до cfg.image_size + случайный горизонтальный флип + ToTensor.
# Валидация/тест: только Resize + ToTensor.
set_seed(42)
# ------------------------------------------------------------
train_transform = T.Compose([
    T.Resize((cfg.image_size, cfg.image_size), interpolation=InterpolationMode.BILINEAR, antialias=True),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),  
])

eval_transform = T.Compose([
    T.Resize((cfg.image_size, cfg.image_size), interpolation=InterpolationMode.BILINEAR, antialias=True),
    T.ToTensor(), 
])

# ------------------------------------------------------------
# преобразования "на лету" через set_transform.
# set_transform передаёт на вход ЧАЩЕ ВСЕГО БАТЧ (dict со списками),
# поэтому функция должна уметь обрабатывать и list, и одиночные объекты.
# ------------------------------------------------------------
def make_set_transform(pipeline: T.Compose):
    def _apply(examples: dict) -> dict:
        imgs = examples["image"]
        # Случай 1: батч (список PIL.Image)
        if isinstance(imgs, list):
            examples["image"] = [pipeline(img) for img in imgs]  # -> список Tensor[C,H,W]
        else:
            # Случай 2: одиночный пример (PIL.Image)
            examples["image"] = pipeline(imgs)                    # -> Tensor[C,H,W]
        return examples
    return _apply

train_ds.set_transform(make_set_transform(train_transform))  # аугментации только для train
val_ds.set_transform(make_set_transform(eval_transform))    
test_ds.set_transform(make_set_transform(eval_transform))   

In [ ]:
# ------------------------------------------------------------
# collate_fn: принимает либо список примеров, либо уже "батч-словарь"
# (что выдаёт HF при auto-collation). Возвращает (images, labels) тензоры.
# ------------------------------------------------------------
def collate_fn(batch):
    # Случай A: HF уже вернул батч-словарь (колонки -> списки/тензоры)
    if isinstance(batch, dict):
        # 'image' после set_transform — либо список Tensor[C,H,W], либо уже Tensor[B,C,H,W]
        imgs = batch["image"]
        if isinstance(imgs, list):
            images = torch.stack(imgs, dim=0)  # [B,C,H,W]
        else:
            images = imgs                      # уже Tensor[B,C,H,W]
        # 'labels' — список ints или Tensor[B]
        labs = batch["labels"]
        labels = torch.as_tensor(labs, dtype=torch.long)
        return images, labels

    # Случай B: обычный список примеров (list of dicts)
    images = torch.stack([b["image"] for b in batch], dim=0)              # [B,C,H,W]
    labels = torch.tensor([b["labels"] for b in batch], dtype=torch.long) # [B]
    return images, labels

set_seed(42)

# ------------------------------------------------------------
# DataLoader'ы
# ------------------------------------------------------------
train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True,  num_workers=0, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=cfg.batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=cfg.batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)


In [ ]:
# ------------------------------------------------------------
# Контрольная проверка: формы, типы и диапазон значений первого батча из train_loader.
# ------------------------------------------------------------
xb, yb = next(iter(train_loader))
print("images:", xb.shape, xb.dtype, f"value range ~ [{xb.min():.3f}, {xb.max():.3f}]")
print("labels:", yb.shape, yb.dtype, "classes:", sorted(set(yb.tolist())))


### Вывод

Контрольная проверка подтверждает корректность подготовки данных:  
- **Форма изображений:** `[16, 3, 128, 128]` соответствует батчу из 64 цветных изображений (3 канала RGB, размер 128×128).  
- **Тип данных:** `torch.float32`, диапазон значений `[0.0, 1.0]` — нормализованный тензор после преобразования `ToTensor()`.  
- **Метки:** размер `[16]`, тип `torch.int64`, классы `[0, 1, 2]` — соответствуют трём категориям датасета (*угловатая пятнистость*, *ржавчина фасоли*, *здоровые листья*).

In [ ]:
# ------------------------------------------------------------
# Определение свёрточной модели для классификации изображений.
# ------------------------------------------------------------
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        # Блок извлечения признаков: Conv-ReLU-MaxPool × 2
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # [B,16,H,W]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # [B,16,H/2,W/2]

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # [B,32,H/2,W/2]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # [B,32,H/4,W/4]
        )
        # Классификатор: сглаживание -> два полносвязных слоя
        self.classifier = nn.Sequential(
            nn.Flatten(),                                # [B, 32*(H/4)*(W/4)]
            nn.Linear(32 * (cfg.image_size // 4) * (cfg.image_size // 4), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.classifier(x)
        return x
set_seed(42)
model = SimpleCNN(num_classes=len(label_names)).to(device)
print(model)


### Подробное описание модели `SimpleCNN`

Модель `SimpleCNN` реализует базовую архитектуру **свёрточной нейронной сети (CNN)**, предназначенной для классификации изображений листьев фасоли по трём категориям:
- *angular_leaf_spot* — угловатая пятнистость листьев,  
- *bean_rust* — ржавчина фасоли,  
- *healthy* — здоровые листья.

---

#### 1. **Блок `features` — извлечение признаков**
Этот блок отвечает за выделение ключевых визуальных особенностей изображений: контуров, текстур, цветовых и структурных паттернов.  

- **`Conv2d(3, 16, kernel_size=3, stride=1, padding=1)`**  
  Первый свёрточный слой принимает RGB-изображение (3 канала) и обучается выделять базовые признаки — края, пятна, прожилки на листьях.  
  Параметры ядра `3×3` и `padding=1` сохраняют пространственный размер.  

- **`ReLU()`**  
  Вводит нелинейность и обнуляет отрицательные значения, позволяя сети обучаться сложным зависимостям.  

- **`MaxPool2d(2)`**  
  Уменьшает размер изображения в 2 раза, выделяя наиболее выраженные признаки и снижая вычислительную нагрузку.  

- **Второй блок `Conv2d(16, 32, ...) + ReLU + MaxPool2d(2)`**  
  Повторение свёртки с увеличением числа каналов до 32 позволяет выявлять более сложные комбинации признаков — например, характерное распределение пятен болезни на листе.  

После двух уровней свёрток и подвыборок размер признаковой карты уменьшается в 4 раза по каждой оси:  
`128×128 → 64×64 → 32×32`, а глубина растёт до 32 каналов.

---

#### 2. **Блок `classifier` — принятие решения**
После извлечения признаков тензор преобразуется в вектор и проходит через полносвязную часть модели.

- **`Flatten()`**  
  Превращает многомерный тензор признаков `[32, 32, 32]` в одномерный вектор длиной 32×32×32 = 32768 для подачи в линейный слой.  

- **`Linear(32768, 16)` + `ReLU()`**  
  Первый полносвязный слой учится находить абстрактные сочетания признаков, которые помогают различать типы заболеваний.  
  Например, он может различать мелкие равномерные пятна (*bean_rust*) и крупные угловатые поражения (*angular_leaf_spot*).

- **`Linear(16, 3)`**  
  Финальный слой выдаёт логиты для трёх классов. После применения `Softmax` на выходе получаем вероятности принадлежности изображения к каждому классу.  

---

#### 3. **Общая структура**
Модель имеет **два уровня свёртки и два уровня классификации**, что делает её лёгкой, интерпретируемой и быстрой для обучения.  
Она идеально подходит для учебных демонстраций, так как:
- позволяет визуализировать влияние оптимизаторов (SGD, Adam и др.) на скорость сходимости;
- даёт наглядное понимание, как CNN извлекает и обобщает признаки изображений.

---

In [ ]:
# ------------------------------------------------------------
# Настройка функции потерь и оптимизатора.
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr)

In [ ]:
# ------------------------------------------------------------
# Тестовый проход одной батч-выборки через модель.
# ------------------------------------------------------------
xb, yb = xb.to(device), yb.to(device)
logits = model(xb)
print("logits:", logits.shape)        # [B, num_classes]
print("loss:", criterion(logits, yb).item())


### Вывод

Проведена проверка прямого прохода батча через модель:

- **Функция потерь:** `CrossEntropyLoss()` — стандартная для многоклассовой классификации; сравнивает распределение логитов модели с правильными метками классов.  
- **Оптимизатор:** `SGD` с начальным шагом обучения `lr = 1e-3`. На следующих этапах этот оптимизатор будет заменяться другими (Momentum, Adam, RMSProp) для сравнения эффективности.

Результаты тестового прохода:
- Размер логитов: `[16, 3]` — для каждого из 16 изображений предсказаны 3 значения (по числу классов).  
- Потеря (`loss ≈ 1.13`) — начальное значение ошибки до обучения, что соответствует случайным прогнозам модели.

Таким образом, прямой проход и вычисление функции потерь выполняются корректно; модель готова к этапу обучения и анализу поведения различных оптимизаторов.


In [ ]:
# ============================================================
# Цикл для сравнения оптимизаторов.
# Предпосылки:
#   - Данные: train_loader, val_loader, test_loader уже созданы.
#   - Список имён классов: label_names.
#   - Устройство: device (cpu/cuda).
# Использование:
#   - Передаём модель и "строитель" оптимизатора (optimizer_builder),
#     чтобы легко менять оптимизатор между запусками.
# ============================================================

from copy import deepcopy
from typing import Callable, Dict, Any
from sklearn.metrics import accuracy_score
import numpy as np

set_seed(42)
@torch.no_grad()
def collect_preds_targets(model: nn.Module, loader: DataLoader):
    model.eval()
    all_true, all_pred, all_prob = [], [], []
    for images, targets in loader:
        images  = images.to(device)
        targets = targets.to(device)
        logits  = model(images)
        probs   = torch.softmax(logits, dim=1)
        preds   = probs.argmax(dim=1)
        all_true.append(targets.cpu())
        all_pred.append(preds.cpu())
        all_prob.append(probs.cpu())
    y_true = torch.cat(all_true).numpy()
    y_pred = torch.cat(all_pred).numpy()
    y_prob = torch.cat(all_prob).numpy()  # [N, num_classes]
    return y_true, y_pred, y_prob


# ------------------------------------------------------------
# Оценка модели: усреднённый loss по элементам и глобальная accuracy (sklearn).
# ------------------------------------------------------------
@torch.no_grad()
def evaluate_model(model: nn.Module, loader: DataLoader, criterion: nn.Module):
    model.eval()
    total_loss, total_items = 0.0, 0
    all_true, all_pred = [], []
    for images, targets in loader:
        images  = images.to(device)
        targets = targets.to(device)
        logits  = model(images)
        loss    = criterion(logits, targets)

        bs = targets.size(0)
        total_loss  += loss.item() * bs
        total_items += bs

        preds = logits.argmax(dim=1)
        all_true.append(targets.cpu())
        all_pred.append(preds.cpu())

    y_true = torch.cat(all_true).numpy()
    y_pred = torch.cat(all_pred).numpy()
    acc = accuracy_score(y_true, y_pred)
    return {"loss": total_loss / max(total_items, 1), "acc": acc}

# ------------------------------------------------------------
# Обучение за одну эпоху: глобальный loss/accuracy по всем элементам (sklearn).
# ------------------------------------------------------------
def train_one_epoch(model: nn.Module, loader: DataLoader, optimizer: torch.optim.Optimizer, criterion: nn.Module):
    model.train()
    total_loss, total_items = 0.0, 0
    all_true, all_pred = [], []
    for images, targets in loader:
        images  = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits = model(images)
        loss   = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        bs = targets.size(0)
        total_loss  += loss.item() * bs
        total_items += bs
        all_true.append(targets.cpu())
        all_pred.append(logits.detach().argmax(dim=1).cpu())

    y_true = torch.cat(all_true).numpy()
    y_pred = torch.cat(all_pred).numpy()
    acc = accuracy_score(y_true, y_pred)
    return {"loss": total_loss / max(total_items, 1), "acc": acc}

In [ ]:
# ------------------------------------------------------------
#  цикл эпох: train → val, журнал history.
# ------------------------------------------------------------
from copy import deepcopy
from typing import Callable, Dict, Any
set_seed(42)
def run_experiment(
    model: nn.Module,
    optimizer_builder: Callable[[Any], torch.optim.Optimizer],
    *,
    num_epochs: int = cfg.epochs,
    print_every: int = 1,
    criterion: nn.Module | None = None,
) -> Dict[str, Any]:
    if criterion is None:
        criterion = nn.CrossEntropyLoss()

    optimizer = optimizer_builder(model.parameters())

    history = {"epoch": [], "train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    for epoch in range(1, num_epochs + 1):
        train_metrics = train_one_epoch(model, train_loader, optimizer, criterion)
        val_metrics   = evaluate_model(model, val_loader,   criterion)

        history["epoch"].append(epoch)
        history["train_loss"].append(train_metrics["loss"])
        history["train_acc"].append(train_metrics["acc"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_acc"].append(val_metrics["acc"])

        if (epoch % print_every) == 0:
            print(f"[epoch {epoch:02d}] "
                  f"train: loss={train_metrics['loss']:.4f}, acc={train_metrics['acc']:.3f} | "
                  f"val:   loss={val_metrics['loss']:.4f}, acc={val_metrics['acc']:.3f}")

    return {"history": history}


In [ ]:
# ============================================================
# Обучение модели с оптимизатором SGD.
# ============================================================
set_seed(42)
model_sgd = SimpleCNN(num_classes=len(label_names)).to(device)
criterion = nn.CrossEntropyLoss()

# Строитель оптимизатора: позволяет переиспользовать общий цикл
def build_sgd(params):
    return torch.optim.SGD(params, lr=cfg.lr)  # базовый SGD без momentum

exp_sgd = run_experiment(
    model=model_sgd,
    optimizer_builder=build_sgd,
    num_epochs=10,
    print_every=1,
    criterion=criterion,
)

# Короткая сводка по итогам валидации
print(f"Лучшее val acc: {max(exp_sgd['history']['val_acc']):.3f}")

In [ ]:
# ------------------------------------------------------------
# Визуализация кривых обучения для SGD.
# ------------------------------------------------------------
import matplotlib.pyplot as plt

epochs = exp_sgd["history"]["epoch"]
tr_loss = exp_sgd["history"]["train_loss"]; va_loss = exp_sgd["history"]["val_loss"]
tr_acc  = exp_sgd["history"]["train_acc"];  va_acc  = exp_sgd["history"]["val_acc"]

plt.figure(figsize=(6,4))
plt.plot(epochs, tr_loss, marker="o", label="train loss")
plt.plot(epochs, va_loss, marker="o", label="val loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("SGD: динамика loss"); plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.plot(epochs, tr_acc, marker="o", label="train acc")
plt.plot(epochs, va_acc, marker="o", label="val acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.title("SGD: динамика accuracy"); plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Тестовая оценка: accuracy + подробный отчёт по классам.
# ------------------------------------------------------------
from sklearn.metrics import classification_report

y_true_test, y_pred_test, y_prob_test = collect_preds_targets(model_sgd, test_loader)
acc_test = accuracy_score(y_true_test, y_pred_test)
print(f"Accuracy (test, SGD): {acc_test:.3f}\n")
print(classification_report(y_true_test, y_pred_test, target_names=label_names, digits=3))


In [ ]:
# ------------------------------------------------------------
# Матрица ошибок и гистограммы precision/recall/F1 по классам.
# ------------------------------------------------------------
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

cm = confusion_matrix(y_true_test, y_pred_test, labels=list(range(len(label_names))))
fig, ax = plt.subplots(figsize=(7,4))
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names).plot(ax=ax, cmap="Blues", values_format="d", colorbar=True)
ax.set_title("Матрица ошибок — SGD (test)")
plt.tight_layout(); plt.show()

prec, rec, f1, support = precision_recall_fscore_support(
    y_true_test, y_pred_test, labels=list(range(len(label_names)))
)

x = np.arange(len(label_names)); w = 0.25
plt.figure(figsize=(7,4))
plt.bar(x - w, prec, width=w, label="precision")
plt.bar(x,       rec, width=w, label="recall")
plt.bar(x + w,    f1, width=w, label="F1")
plt.xticks(x, label_names); plt.ylim(0, 1)
plt.ylabel("score"); plt.title("Качество по классам — SGD (test)")
plt.grid(axis="y", linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Saliency map: |∂ logit_true / ∂ input|, показываем как тепловую карту.
# ============================================================

def saliency_map(model: nn.Module, images: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    """
    Возвращает Карту внимания (saliency-карту) для каждого изображения батча:
    """
    model.eval()
    images = images.clone().detach().to(device).requires_grad_(True)  # включаем градиенты по входу
    labels = labels.to(device)

    logits = model(images)                      # [B, num_classes]
    selected = logits.gather(1, labels.view(-1,1)).squeeze(1)  # логит «правильного» класса
    selected.backward(torch.ones_like(selected))                

    grad = images.grad.detach()                
    sal = grad.abs().max(dim=1).values          
    B = sal.size(0)
    sal = sal.view(B, -1)
    sal = (sal - sal.min(dim=1, keepdim=True).values) / (sal.max(dim=1, keepdim=True).values - sal.min(dim=1, keepdim=True).values + 1e-8)
    sal = sal.view(B, images.size(2), images.size(3))
    return sal.cpu()

# берём небольшой батч из валидации
images_demo, labels_demo = next(iter(val_loader))
images_demo = images_demo[:8]
labels_demo = labels_demo[:8]
sal_demo = saliency_map(model_sgd, images_demo, labels_demo)  # [B,H,W]

# Визуализация: исходное изображение + saliency (как тепловая карта)
import matplotlib.pyplot as plt
B = images_demo.size(0)
cols = 2
rows = B
plt.figure(figsize=(cols*3, rows*2.2))
for i in range(B):
    # Исходное
    ax = plt.subplot(rows, cols, 2*i+1)
    ax.imshow(images_demo[i].permute(1,2,0).numpy(), interpolation="none")
    ax.set_title(f"{label_names[int(labels_demo[i])]}", fontsize=9); ax.axis("off")

    # Saliency
    ax = plt.subplot(rows, cols, 2*i+2)
    ax.imshow(sal_demo[i].numpy(), cmap="inferno", interpolation="none")
    ax.set_title("saliency", fontsize=9); ax.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Модель  и обучение с SGD + momentum.
# ------------------------------------------------------------
set_seed(42)
model_momentum = SimpleCNN(num_classes=len(label_names)).to(device)
criterion = nn.CrossEntropyLoss()

def build_sgd_momentum(params):
    return torch.optim.SGD(params, lr=cfg.lr, momentum=0.9)  # nesterov=False (классический momentum)

exp_momentum = run_experiment(
    model=model_momentum,
    optimizer_builder=build_sgd_momentum,
    num_epochs=10,
    print_every=1,
    criterion=criterion,
)

# Короткая сводка по итогам валидации
print(f"Лучшее val acc: {max(exp_momentum['history']['val_acc']):.3f}")

In [ ]:
# ------------------------------------------------------------
# Визуализация кривых обучения для SGD+Momentum.
# ------------------------------------------------------------
import matplotlib.pyplot as plt

ep = exp_momentum["history"]["epoch"]
tr_loss = exp_momentum["history"]["train_loss"]; va_loss = exp_momentum["history"]["val_loss"]
tr_acc  = exp_momentum["history"]["train_acc"];  va_acc  = exp_momentum["history"]["val_acc"]

plt.figure(figsize=(6,4))
plt.plot(ep, tr_loss, marker="o", label="train loss")
plt.plot(ep, va_loss, marker="o", label="val loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("SGD+Momentum: динамика loss")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.plot(ep, tr_acc, marker="o", label="train acc")
plt.plot(ep, va_acc, marker="o", label="val acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.title("SGD+Momentum: динамика accuracy")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Тестовая оценка (sklearn): accuracy + подробный отчёт по классам.
# ------------------------------------------------------------
from sklearn.metrics import accuracy_score, classification_report

y_true_m, y_pred_m, y_prob_m = collect_preds_targets(model_momentum, test_loader)
acc_test_m = accuracy_score(y_true_m, y_pred_m)
print(f"Accuracy (test, SGD+Momentum): {acc_test_m:.3f}\n")
print(classification_report(y_true_m, y_pred_m, target_names=label_names, digits=3))


In [ ]:
# ------------------------------------------------------------
# Матрица ошибок и гистограммы precision/recall/F1 по классам (sklearn).
# ------------------------------------------------------------
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import numpy as np

cm_m = confusion_matrix(y_true_m, y_pred_m, labels=list(range(len(label_names))))
fig, ax = plt.subplots(figsize=(5,4))
ConfusionMatrixDisplay(confusion_matrix=cm_m, display_labels=label_names).plot(ax=ax, cmap="Blues", values_format="d", colorbar=True)
ax.set_title("Матрица ошибок — SGD+Momentum (test)")
plt.tight_layout(); plt.show()

prec_m, rec_m, f1_m, sup_m = precision_recall_fscore_support(
    y_true_m, y_pred_m, labels=list(range(len(label_names)))
)

x = np.arange(len(label_names)); w = 0.25
plt.figure(figsize=(7,4))
plt.bar(x - w, prec_m, width=w, label="precision")
plt.bar(x,       rec_m, width=w, label="recall")
plt.bar(x + w,    f1_m, width=w, label="F1")
plt.xticks(x, label_names); plt.ylim(0, 1)
plt.ylabel("score"); plt.title("Качество по классам — SGD+Momentum (test)")
plt.grid(axis="y", linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Saliency map: |∂ logit_true / ∂ input|, показываем как тепловую карту.
# ============================================================

images_demo, labels_demo = next(iter(val_loader))
images_demo = images_demo[:8]
labels_demo = labels_demo[:8]
sal_demo = saliency_map(model_momentum, images_demo, labels_demo)  # [B,H,W]

# Визуализация: исходное изображение + saliency (как тепловая карта)
import matplotlib.pyplot as plt
B = images_demo.size(0)
cols = 2
rows = B
plt.figure(figsize=(cols*3, rows*2.2))
for i in range(B):
    # Исходное
    ax = plt.subplot(rows, cols, 2*i+1)
    ax.imshow(images_demo[i].permute(1,2,0).numpy(), interpolation="none")
    ax.set_title(f"{label_names[int(labels_demo[i])]}", fontsize=9); ax.axis("off")

    # Saliency
    ax = plt.subplot(rows, cols, 2*i+2)
    ax.imshow(sal_demo[i].numpy(), cmap="inferno", interpolation="none")
    ax.set_title("saliency", fontsize=9); ax.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Модель и обучение с Adam.
# ------------------------------------------------------------
set_seed(42)
model_adam = SimpleCNN(num_classes=len(label_names)).to(device)
criterion = nn.CrossEntropyLoss()

def build_adam(params):
    return torch.optim.Adam(params, lr=cfg.lr)  # Может не обучаться, придется подбирать экспериментально

exp_adam = run_experiment(
    model=model_adam,
    optimizer_builder=build_adam,
    num_epochs=10,
    print_every=1,
    criterion=criterion,
)

print(f"Лучшее val acc: {max(exp_adam['history']['val_acc']):.3f}")

In [ ]:
# ------------------------------------------------------------
# Визуализация кривых обучения для Adam.
# ------------------------------------------------------------
import matplotlib.pyplot as plt

ep = exp_adam["history"]["epoch"]
tr_loss = exp_adam["history"]["train_loss"]; va_loss = exp_adam["history"]["val_loss"]
tr_acc  = exp_adam["history"]["train_acc"];  va_acc  = exp_adam["history"]["val_acc"]

plt.figure(figsize=(6,4))
plt.plot(ep, tr_loss, marker="o", label="train loss")
plt.plot(ep, va_loss, marker="o", label="val loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("Adam: динамика loss")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.plot(ep, tr_acc, marker="o", label="train acc")
plt.plot(ep, va_acc, marker="o", label="val acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.title("Adam: динамика accuracy")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Тестовая оценка (sklearn): accuracy + подробный отчёт по классам.
# ------------------------------------------------------------
from sklearn.metrics import accuracy_score, classification_report

y_true_a, y_pred_a, y_prob_a = collect_preds_targets(model_adam, test_loader)
acc_test_a = accuracy_score(y_true_a, y_pred_a)
print(f"Accuracy (test, Adam): {acc_test_a:.3f}\n")
print(classification_report(y_true_a, y_pred_a, target_names=label_names, digits=3))


In [ ]:
# ------------------------------------------------------------
# Матрица ошибок и гистограммы precision/recall/F1 по классам (sklearn).
# ------------------------------------------------------------
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import numpy as np

cm_a = confusion_matrix(y_true_a, y_pred_a, labels=list(range(len(label_names))))
fig, ax = plt.subplots(figsize=(5,4))
ConfusionMatrixDisplay(confusion_matrix=cm_a, display_labels=label_names).plot(ax=ax, cmap="Blues", values_format="d", colorbar=True)
ax.set_title("Матрица ошибок — Adam (test)")
plt.tight_layout(); plt.show()

prec_a, rec_a, f1_a, sup_a = precision_recall_fscore_support(
    y_true_a, y_pred_a, labels=list(range(len(label_names)))
)

x = np.arange(len(label_names)); w = 0.25
plt.figure(figsize=(7,4))
plt.bar(x - w, prec_a, width=w, label="precision")
plt.bar(x,       rec_a, width=w, label="recall")
plt.bar(x + w,    f1_a, width=w, label="F1")
plt.xticks(x, label_names); plt.ylim(0, 1)
plt.ylabel("score"); plt.title("Качество по классам — Adam (test)")
plt.grid(axis="y", linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Saliency map: |∂ logit_true / ∂ input|, показываем как тепловую карту.
# ============================================================

images_demo, labels_demo = next(iter(val_loader))
images_demo = images_demo[:8]
labels_demo = labels_demo[:8]
sal_demo = saliency_map(model_adam, images_demo, labels_demo)  # [B,H,W]

# Визуализация: исходное изображение + saliency (как тепловая карта)
import matplotlib.pyplot as plt
B = images_demo.size(0)
cols = 2
rows = B
plt.figure(figsize=(cols*3, rows*2.2))
for i in range(B):
    # Исходное
    ax = plt.subplot(rows, cols, 2*i+1)
    ax.imshow(images_demo[i].permute(1,2,0).numpy(), interpolation="none")
    ax.set_title(f"{label_names[int(labels_demo[i])]}", fontsize=9); ax.axis("off")

    # Saliency
    ax = plt.subplot(rows, cols, 2*i+2)
    ax.imshow(sal_demo[i].numpy(), cmap="inferno", interpolation="none")
    ax.set_title("saliency", fontsize=9); ax.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Модель (новый экземпляр) и обучение с AdamW.
# AdamW = Adam + корректная L2-регуляризация (weight_decay).
# ------------------------------------------------------------
set_seed(42)
model_adamw = SimpleCNN(num_classes=len(label_names)).to(device)
criterion = nn.CrossEntropyLoss()

def build_adamw(params):
    return torch.optim.AdamW(params, lr=1e-3, weight_decay=1e-2) 

exp_adamw = run_experiment(
    model=model_adamw,
    optimizer_builder=build_adamw,
    num_epochs=10,
    print_every=1,
    criterion=criterion,
)
print(f"Лучшее val acc: {max(exp_adamw['history']['val_acc']):.3f}")

In [ ]:
# ------------------------------------------------------------
# Визуализация кривых обучения для AdamW.
# ------------------------------------------------------------
import matplotlib.pyplot as plt

ep = exp_adamw["history"]["epoch"]
tr_loss = exp_adamw["history"]["train_loss"]; va_loss = exp_adamw["history"]["val_loss"]
tr_acc  = exp_adamw["history"]["train_acc"];  va_acc  = exp_adamw["history"]["val_acc"]

plt.figure(figsize=(6,4))
plt.plot(ep, tr_loss, marker="o", label="train loss")
plt.plot(ep, va_loss, marker="o", label="val loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("AdamW: динамика loss")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.plot(ep, tr_acc, marker="o", label="train acc")
plt.plot(ep, va_acc, marker="o", label="val acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.title("AdamW: динамика accuracy")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ------------------------------------------------------------
# Тестовая оценка (sklearn): accuracy + подробный отчёт по классам.
# ------------------------------------------------------------
from sklearn.metrics import accuracy_score, classification_report

y_true_w, y_pred_w, y_prob_w = collect_preds_targets(model_adamw, test_loader)
acc_test_w = accuracy_score(y_true_w, y_pred_w)
print(f"Accuracy (test, AdamW): {acc_test_w:.3f}\n")
print(classification_report(y_true_w, y_pred_w, target_names=label_names, digits=3))


In [ ]:
# ------------------------------------------------------------
# Матрица ошибок и гистограммы precision/recall/F1 по классам (sklearn).
# ------------------------------------------------------------
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import numpy as np

cm_w = confusion_matrix(y_true_w, y_pred_w, labels=list(range(len(label_names))))
fig, ax = plt.subplots(figsize=(5,4))
ConfusionMatrixDisplay(confusion_matrix=cm_w, display_labels=label_names).plot(ax=ax, cmap="Blues", values_format="d", colorbar=True)
ax.set_title("Матрица ошибок — AdamW (test)")
plt.tight_layout(); plt.show()

prec_w, rec_w, f1_w, sup_w = precision_recall_fscore_support(
    y_true_w, y_pred_w, labels=list(range(len(label_names)))
)

x = np.arange(len(label_names)); w = 0.25
plt.figure(figsize=(7,4))
plt.bar(x - w, prec_w, width=w, label="precision")
plt.bar(x,       rec_w, width=w, label="recall")
plt.bar(x + w,    f1_w, width=w, label="F1")
plt.xticks(x, label_names); plt.ylim(0, 1)
plt.ylabel("score"); plt.title("Качество по классам — AdamW (test)")
plt.grid(axis="y", linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Saliency map: |∂ logit_true / ∂ input|, показываем как тепловую карту.
# ============================================================

images_demo, labels_demo = next(iter(val_loader))
images_demo = images_demo[:8]
labels_demo = labels_demo[:8]
sal_demo = saliency_map(model_adamw, images_demo, labels_demo)  # [B,H,W]

# Визуализация: исходное изображение + saliency (как тепловая карта)
import matplotlib.pyplot as plt
B = images_demo.size(0)
cols = 2
rows = B
plt.figure(figsize=(cols*3, rows*2.2))
for i in range(B):
    # Исходное
    ax = plt.subplot(rows, cols, 2*i+1)
    ax.imshow(images_demo[i].permute(1,2,0).numpy(), interpolation="none")
    ax.set_title(f"{label_names[int(labels_demo[i])]}", fontsize=9); ax.axis("off")

    # Saliency
    ax = plt.subplot(rows, cols, 2*i+2)
    ax.imshow(sal_demo[i].numpy(), cmap="inferno", interpolation="none")
    ax.set_title("saliency", fontsize=9); ax.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Сравнение динамики обучения для нескольких оптимизаторов.


import matplotlib.pyplot as plt

def _collect_histories() -> list[tuple[str, dict]]:
    """Собирает пары (название, history) из глобальных переменных, если они существуют."""
    candidates = [
        ("SGD",       globals().get("exp_sgd",       {}).get("history")),
        ("Momentum",  globals().get("exp_momentum",  {}).get("history")),
        ("Adam",      globals().get("exp_adam",      {}).get("history")),
        ("AdamW",     globals().get("exp_adamw",     {}).get("history")),
    ]
    return [(name, hist) for name, hist in candidates if isinstance(hist, dict) and len(hist.get("epoch", [])) > 0]

histories = _collect_histories()
assert histories, "Нет историй обучения для сравнения. Убедитесь, что exp_* уже получены."

# Приводим все серии к общей длине (минимум эпох среди переданных историй),
# чтобы кривые были сопоставимы по оси X.
min_len = min(len(h["epoch"]) for _, h in histories)

# Готовим наборы серий для отрисовки
series = []
for name, h in histories:
    series.append({
        "name": name,
        "epoch":     h["epoch"][:min_len],
        "train_loss":h["train_loss"][:min_len],
        "val_loss":  h["val_loss"][:min_len],
        "train_acc": h["train_acc"][:min_len],
        "val_acc":   h["val_acc"][:min_len],
    })

# --- Loss ---
plt.figure(figsize=(7, 4.2))
for s in series:
    plt.plot(s["epoch"], s["train_loss"], marker="o", linewidth=1.5, label=f"{s['name']} | train")
    plt.plot(s["epoch"], s["val_loss"],   marker="o", linewidth=1.5, linestyle="--", label=f"{s['name']} | val")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("Сравнение оптимизаторов: loss")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(ncol=2, fontsize=9)
plt.tight_layout(); plt.show()

# --- Accuracy ---
plt.figure(figsize=(7, 4.2))
for s in series:
    plt.plot(s["epoch"], s["train_acc"], marker="o", linewidth=1.5, label=f"{s['name']} | train")
    plt.plot(s["epoch"], s["val_acc"],   marker="o", linewidth=1.5, linestyle="--", label=f"{s['name']} | val")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.title("Сравнение оптимизаторов: accuracy")
plt.grid(True, linestyle="--", alpha=0.5); plt.legend(ncol=2, fontsize=9)
plt.tight_layout(); plt.show()


In [ ]:
# ============================================================
# Сводная таблица финальных метрик на тесте (sklearn accuracy).
# ============================================================

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

def _ensure_preds(model_var_name: str, loader, cache_prefix: str):
    """
    Проверяет, есть ли уже y_true_*, y_pred_*; если нет — собирает через collect_preds_targets().
    Возвращает (y_true, y_pred).
    """
    y_true_name = f"y_true_{cache_prefix}"
    y_pred_name = f"y_pred_{cache_prefix}"

    if y_true_name in globals() and y_pred_name in globals():
        return globals()[y_true_name], globals()[y_pred_name]

    model_obj = globals().get(model_var_name, None)
    assert model_obj is not None, f"Модель {model_var_name} не найдена. Сначала запустите её обучение."
    y_true, y_pred, _ = collect_preds_targets(model_obj, loader)
    globals()[y_true_name] = y_true
    globals()[y_pred_name] = y_pred
    return y_true, y_pred

rows = []
# Описываем, что сравниваем: (метка, имя модели в глобалах, префикс для кэша, история)
candidates = [
    ("SGD",       "model_sgd",      "test", globals().get("exp_sgd",      {}).get("history")),
    ("Momentum",  "model_momentum", "m",    globals().get("exp_momentum", {}).get("history")),
    ("Adam",      "model_adam",     "a",    globals().get("exp_adam",     {}).get("history")),
    ("AdamW",     "model_adamw",    "w",    globals().get("exp_adamw",    {}).get("history")),
]

for name, model_glob, cache_pref, hist in candidates:
    if model_glob not in globals() or not isinstance(hist, dict):
        continue
    y_true, y_pred = _ensure_preds(model_glob, test_loader, cache_pref)
    acc = accuracy_score(y_true, y_pred)
    best_val = float(np.max(hist["val_acc"])) if len(hist.get("val_acc", [])) else np.nan
    rows.append({"optimizer": name, "test_acc": acc, "best_val_acc": best_val})

assert rows, "Нет данных для таблицы. Убедитесь, что эксперименты запущены."
df_cmp = pd.DataFrame(rows).sort_values("test_acc", ascending=False).reset_index(drop=True)

display(
    df_cmp.style
    .set_caption("Сводная таблица: test accuracy и лучшая val accuracy")
    .format({"test_acc": "{:.3f}", "best_val_acc": "{:.3f}"})
    .hide(axis="index")
)
